# Introduction to TM-Databases in Computational Chemistry

This notebook introduces how to visualize and analyze computational datasets (DSs) within the field of computational chemistry. We will focus on a dataset containing transition metal complexes.

In this notebook, we will:
- Visualize molecular properties of key quantum mechanical properties of transition metal complexes.
- Explore the chemical space of a metal-containing designed dataset.
- Detect potential errors. 

**NB!**
We need to install and import some libraries (most(of all) of them are already installed)

**NB!**
    📂 Note:
    To run this notebook successfully, make sure you download the required data files from the GitHub repository: https://github.com/lmoranglez/CAMLC25_DB.git and save them in the same folder as this notebook.

Chemistry & Structure Libraries:
- rdkit (for molecule handling and visualization)
- py3Dmol (for 3D visualization)
- xyz2mol (convert XYZ files to RDKit molecules)
- morfeus-ml (steric/electronic descriptors)
- xtb (semi-empirical quantum chemistry)
- molSimplify (complex generation)

Data Science & Visualization
- pandas (dataframes)
- numpy (numerical operations)
- matplotlib (plotting)
- seaborn (statistical plots)
- plotly (interactive plots)
- scikit-learn (PCA, machine learning)

In [ ]:
# Chemistry libraries
!pip install py3Dmol
!pip install rdkit

# Data science & visualization
!pip install pandas numpy matplotlib seaborn plotly scikit-learn


In [ ]:
# Visualize the libraries in your ipython kernel
!pip list

## 1. Dataset of TMC complexes: tmQM dataset

We will explore the content of an available dataset [tmQM](https://github.com/uiocompcat/tmQM). \
As explained before, this dataaset contains TMC with a variety of TM, ligands, charges {-1,0,1}, ..., extracted from the Cambridge Structural Database and later computed at the DFT(TPSSh-D3BJ/def2-SVP) level and their SMILES. The polarizability is at the GFN2-xTB level of theory.

**NB!**
    📂 Data availability: \
Dataset has to be downloaded!

To prevent space related problems, only download these two files/folders. 

-> Download the materials [file to download](https://github.com/uiocompcat/tmQM/blob/master/tmQM/tmQM_y.csv)  \
-> Clone the entire repository in one separated cell **git clone https://github.com/uiocompcat/tmQM.git**

In [1]:
# ensures plots are shown within the notebook
import pandas as pd
from rdkit.Chem import Draw
from sklearn.decomposition import PCA
from pathlib import Path
import os 

# plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

# --- Chemical Informatics (RDKit) ---
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors, AllChem #, rdMolDescriptors, rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole

from pathlib import Path

import py3Dmol

### 1.1. Explore the dataset
✏️  Data points and properties

In [ ]:
# load QM properties and SMILES
data = pd.read_csv('tmQM_y.csv', sep=';')

# number of TMCs
print(f'Number of TMCs: {len(data)}')

data.head()

### 1.2. Distribution of the properties
✏️  Overview of the properties: basic statistics and distribution analysis



In [ ]:
# descriptive statistics (count, mean, std, min, max, quartiles)
data.describe().round(3)

In [ ]:
# visualize the properties adding the describe statistics
y_to_show = data.select_dtypes(include=['float64', 'int64'])  # Select only numeric columns
y_to_show = y_to_show.columns.tolist()

# plot grid
fig, axes = plt.subplots(4, 2, figsize=(15, 15))
axes = axes.flatten()  # Flatten to 1D array

color_palette = sns.color_palette("husl", len(y_to_show))

for i, y_prop in enumerate(y_to_show):

    sns.histplot(data[y_prop], 
                 bins=40, 
                 kde=True, 
                 ax=axes[i],
                 color=color_palette[i])
    
    # add the mean as a vertical line and the std as a shaded area
    mean = data[y_prop].mean()
    std = data[y_prop].std()
    axes[i].axvline(mean, color='red', linestyle='--', label='Mean')
    axes[i].fill_betweenx([0, axes[i].get_ylim()[1]], mean - std, mean + std, 
                          color='grey', alpha=0.2, label='1 Std Dev')
    axes[i].legend()

    axes[i].set_title(f'Distribution of {y_prop}')
    axes[i].set_xlabel(y_prop)
    axes[i].set_ylabel('Frequency')
    
plt.tight_layout()

### 1.3. Visualize the chemical space: any correlation?
Pairplot distribution and PCA display.

Define a property to use as **x-axis** in the pairplot.
Try the following:

- Polarizability
- Electronic_E
- HL_Gap

In [5]:
# property to visualize as in the dataframe
tmqm_property = 'Polarizability'

In [ ]:
exclude_props = ['CSD_years', 'SMILES', tmqm_property]
all_props = [col for col in data.columns if col not in exclude_props and data[col].dtype in ['float64', 'int64']]
y_props = all_props

# plot grid
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()  # Flatten to 1D array
for i, y_prop in enumerate(y_props):

    ax = axes[i]
    sns.scatterplot(x=data[tmqm_property], y=data[y_prop], 
                    ax=ax, 
                    s=10, 
                    color='skyblue', 
                    alpha=0.5)
    ax.set_xlabel(tmqm_property)
    ax.set_ylabel(y_prop)
    ax.set_title(f'{tmqm_property} vs {y_prop}')

# Hide unused axes if less than 6 y_props
for j in range(len(y_props), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

#### Visualizing with PCA

In [7]:
n_components = 5  # Number of PCA components to compute
component_x = 0
component_y = 3

# Select numeric columns and drop NaNs
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
X = data[numeric_cols].dropna()

# Run PCA
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

In [ ]:
# plot the principal components
fig, axes = plt.subplots(3, 3, figsize=(18, 18))
axes = axes.flatten()

for i, prop in enumerate(numeric_cols):
    ax = axes[i]
    sc = ax.scatter(X_pca[:, component_x], X_pca[:, component_y], s=10, c=X[prop], cmap='viridis', alpha=0.5)
    ax.set_xlabel(f'PC{component_x + 1} ({pca.explained_variance_ratio_[component_x]:.2%} variance)')
    ax.set_ylabel(f'PC{component_y + 1} ({pca.explained_variance_ratio_[component_y]:.2%} variance)')
    ax.set_title(f'PCA colored by: {prop}')
    plt.colorbar(sc, ax=ax, label=prop)

# Hide unused axes if fewer than 9 properties
for j in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[j])

#### Interactive PCA-plot with plotly library

In [ ]:
# Select numeric columns and drop NaNs
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
X = data[numeric_cols].dropna()

# Create a DataFrame for Plotly
component_names = [f'PC{i+1}' for i in range(n_components)]
pca_df = pd.DataFrame(X_pca, columns=component_names)
pca_df[f'{tmqm_property}'] = X[f'{tmqm_property}'].reset_index(drop=True)
pca_df['index'] = X.index.values  # Add index for hover

# Interactive plot for Electronic_E only
fig = px.scatter(
    pca_df, x=f'PC{component_x + 1}', y=f'PC{component_y + 1}',
    color=f'{tmqm_property}',
    color_continuous_scale='viridis',
    hover_data=['index', f'{tmqm_property}'],
    title=f'PCA colored by: {tmqm_property}',
    labels={
        'PC1': f'PC{component_x + 1} ({pca.explained_variance_ratio_[component_x]:.2%} variance)',
        'PC2': f'PC{component_y + 1} ({pca.explained_variance_ratio_[component_y]:.2%} variance)',
        f'{tmqm_property}': f'{tmqm_property}'
    }
)

fig.show()

#### Analysis of out-of-the-box points
[CSD search](https://www.ccdc.cam.ac.uk/structures/)

In [ ]:
id = 23 ### int nunber to be filled
data.iloc[id]
# compare with the rest of the values

### 1.4. Analysis of chemical structures on the distribution tails

#### Visualize the "extreme" chemical structures according to their property
The dataset has metal-containing SMILES. These are derived from Hückel theory as described in this [preprint](link). 

**NB!** Be aware of the difficulties of the TM complexes with SMILES. 

In [14]:
# Define the property and the number of structures to display in the extremes (max and min) values.
n_items = 5
tmqm_property = 'Polarizability' 

In [15]:
# functions to get the indices of the molecules with max and min property values and visualize them
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.info')

def visualize_molecule(idx_to_visualize, data):
    """
    Visualize a list of molecules as 2D structures using their SMILES.

    Args:
        idx_to_visualize (list of int): List of DataFrame indices for the molecules to visualize.
        data (pd.DataFrame): DataFrame containing a 'SMILES' column with molecular SMILES strings.

    Returns:
        None. Displays a grid image of the molecules in the notebook.
    """
    
    if idx_to_visualize:
        mols_to_visualize = []
        for idx in idx_to_visualize:
            smiles = data.loc[idx, 'SMILES']
            mol = Chem.MolFromSmiles(smiles)

            if mol is not None:
                try:
                    mol_with_h = Chem.AddHs(mol)
                    AllChem.EmbedMolecule(mol_with_h)   # is it required?
                    #AllChem.MMFFOptimizeMolecule(mol_with_h)     # is it required?
                    Draw.PrepareMolForDrawing(mol_with_h)
                    mols_to_visualize.append(mol_with_h)
                    
                except Exception as e:
                    print(f"Error processing molecule at index {idx}: {e}")
            else:
                print(f"Invalid SMILES at index {idx}: {smiles}")

        img = Draw.MolsToGridImage(mols_to_visualize,       # Display molecules horizontally in a grid
                                   molsPerRow=len(mols_to_visualize), 
                                   subImgSize=(350, 350))
        
        print(f"Visualizing {len(mols_to_visualize)} molecules")
        display(img)
    else:
        print("No valid indices to visualize.")

def get_z_score(data, property_name):
    """
    Calculate the z-score for a given property in the DataFrame.

    Args:
        data (pd.DataFrame): The input DataFrame containing the data.
        property_name (str): The column name for which to calculate z-scores.

    Returns:
        pd.Series: The z-scores for the specified property.
    """

    mean = data[property_name].mean()
    std = data[property_name].std()
    z_scores = (data[property_name] - mean) / std
    
    return z_scores

In [ ]:
# filter valid TM-SMILES
valid_tm_smiles = data['SMILES'].notna().sum()
nonvalid_tm_smiles = data['SMILES'].isna().sum()

print(f"Valid TM SMILES: {valid_tm_smiles} out of {len(data)}")
print(f"Non-valid TM SMILES: {nonvalid_tm_smiles} out of {len(data)}")

idx_valid_tm_smiles = data.index[data['SMILES'].notna()].tolist()
idx_nonvalid_tm_smiles = data.index[data['SMILES'].isna()].tolist()

##### Find the most different chemical structures: **z-score**
Formula to standarize data and calculate the dispersion of the datapoints \
            $z = (x — μ) / σ​$

In [17]:
def get_idx_z_scores(data, idx_valid_tm_smiles, property_name, n_items):
    """
    Get indices of molecules with the highest and lowest z-scores for a given property.

    Args:
        data (pd.DataFrame): The input DataFrame containing the data.
        idx_valid_tm_smiles (list of int): Indices of rows with valid SMILES
        property_name (str): The column name for which to calculate z-scores.
        n_items (int): Number of molecules to select from each tail (max/min).

    Returns:
        max_z_idx (list of int): Indices of molecules with the highest z-scores.
        min_z_idx (list of int): Indices of molecules with the lowest z-scores.
    """
    
    z_scores = get_z_score(data, property_name)
    data['z_score'] = z_scores
    #z_scores = data['z_score']
    
    # get the n_items with the highest and lowest z-scores
    max_z_scores = z_scores.loc[idx_valid_tm_smiles].nlargest(n_items)
    min_z_scores = z_scores.loc[idx_valid_tm_smiles].nsmallest(n_items)

    max_z_idx = max_z_scores.index.tolist()
    min_z_idx = min_z_scores.index.tolist()
    
    return max_z_idx, min_z_idx

In [ ]:
# get the n_items molecules with the highest and lowest z-scores for the selected property
max_z_idx, min_z_idx = get_idx_z_scores(data, idx_valid_tm_smiles, tmqm_property, n_items)

print("Max z-score indices and values:", [(idx, str(data.loc[idx, 'z_score'])) for idx in max_z_idx])
visualize_molecule(max_z_idx, data)
print('---'*20)
print("Min z-score indices and values:", [(idx, str(data.loc[idx, 'z_score'])) for idx in min_z_idx])
visualize_molecule(min_z_idx, data)

#### Visualize the previous structures within the chemical design space 

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for i, y_prop in enumerate(y_props):
    ax = axes[i]

    sns.scatterplot(x=data[tmqm_property], y=data[y_prop], ax=ax, s=10, color='lightblue', alpha=0.5)
    
    # Highlight max indices
    ax.scatter(data.loc[max_z_idx, tmqm_property], data.loc[max_z_idx, y_prop],
               color='violet', s=60, label='Max', edgecolor='black', zorder=10)
    # Highlight min indices
    ax.scatter(data.loc[min_z_idx, tmqm_property], data.loc[min_z_idx, y_prop],
               color='orange', s=60, label='Min', edgecolor='black', zorder=10)
    
    ax.set_xlabel(tmqm_property)
    ax.set_ylabel(y_prop)
    ax.set_title(f'{tmqm_property} vs {y_prop}')
    ax.legend(loc='best')

for j in range(len(y_props), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

#### Visualize the previous structures within the PCA-plot 

In [20]:
n_components = 5  # Number of PCA components to compute
component_x = 0
component_y = 1

In [ ]:
# visualize PCA
fig, axes = plt.subplots(3, 3, figsize=(18, 18))  # Adjust grid size as needed
axes = axes.flatten()

for i, prop in enumerate(numeric_cols):
    ax = axes[i]
    
    sc = ax.scatter(X_pca[:,0], X_pca[:,1], s=10, c=X[prop], cmap='viridis', alpha=0.5)


    ax.scatter(X_pca[max_z_idx,0], X_pca[max_z_idx,1],
               color='violet', s=60, label='Max', edgecolor='black', zorder=10)
    ax.scatter(X_pca[min_z_idx,0], X_pca[min_z_idx,1],
               color='orange', s=60, label='Min', edgecolor='black', zorder=10)
    
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    ax.set_title(f'PCA colored by: {prop}')
    ax.legend(loc='best')
    plt.colorbar(sc, ax=ax, label=prop)

# Hide unused axes if fewer than grid size
for j in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

--------------------------------------------------------------------------------

## 2. Designing a defined chemical space: metal-containing DS

The dataset will contain complexes of a single transition metal centre. For this study, we select **Fe** as **metal centre**. 

Different **ligands** are considered to ensure a varied environment around the metal centre. 

- *Neutral O-/N-donors:* water, ammonia, methylamine
- *Anionic donors:* chloride (hard), cyanide (soft)
- *Aromatic N-donor:* pyridine

We consider a broad range of common **coordination numbers** (the number of atoms directly bonded to the metal center) for transition metals, from 3 to 8.

For each coordination number, we define possible **coordination geometries**.


**NB!**
    📂 Data availability:

The full dataset generated by enumerating all combinations of the above variables is **computationally expensive** to generate and results in a very large file. \
Thus, pre-computed dataset (Fe + the ligands listed above) is provided as a ready-to-use `.csv` file in this [GitHub repository](https://github.com/lmoranglez/metaldb_camlc.git)

### DO NOT RUN THE FOLLOWING CELLS!! 👀

### 2.1. Generation of the dataset

In [ ]:
!pip install molSimplify
!conda install -c conda-forge xtb xyz2mol -y
!pip install morfeus-ml # install morfeus with pip, not recognized with conda 

In [ ]:
from morfeus import Sterimol, BuriedVolume, ConeAngle, SolidAngle, SASA, XTB, read_xyz
from molSimplify.Scripts.generator import startgen_pythonic
#from molSimplify.Scripts.io import name_complex
import shutil
import random
import re
from xyz2mol import xyz2mol

In [24]:
# variables of our chemical space
metals = 'Fe'
ligands = ['water', 'ammonia', 'chloride', 'cyanide', 'methylamine', 'pyridine']

coordination_numbers = [3, 4, 5, 6, 7, 8]  # Focus on common coordination numbers for these metals
geometries = {
    3: ['tpl'],  # 'trigonal_planar' 
    4: ['thd', 'sqp'], # tetrahedral, square planar
    5: ['tbp', 'spy'], # trigonal_bipyramidal, square_pyramidal
    6: ['oct', 'tpr'], # octahedral, trigonal_prismatic
    7: ['pbp'], # pentagonal_bipyramidal
    8: ['sqap', 'tdhd'] # square_antiprismatic, trigonal_dodecahedral
}

In [ ]:
# create a chemical space varying all possible combinations from the code above
# run molSimplify

# Potential issues with Windows users
for cn in coordination_numbers:

    for geo in geometries[cn]:
    
        for i in range(5):  # Generate 10 random complexes for each geometry

            ligands_for_complex = random.choices(ligands, k=cn)
            print(ligands_for_complex, type(ligands_for_complex))

            # ligands_for complex needs to be a string of ligands separated by commas
            ligands_for_complex = ','.join(ligands_for_complex)

            input_dict = {
                '-core': metals,
                '-coord': str(cn),
                '-geometry': geo,
                '-lig': ligands_for_complex,
                '-skipANN': 'False',
                #'opt': True,
                #'-outdir': 'generated_complexes',
                #'-filename': f'complex_{metals}_{geo}_{str(i)}.xyz'
            }

            startgen_pythonic(input_dict, write=True)

In [13]:
# extract information for each file
dataset_info = []

def parse_molinp_file(molinp_path):
    """
    Parse specific fields from a .molinp file.

    Args:
        molinp_path (str or Path): Path to the .molinp input file.

    Returns:
        dict: Dictionary with keys for each parsed field (e.g., 'core', 'coord', 'charge', etc.).
    """

    target_fields = {
        'core': None,
        'oxstate': None, 
        'coord': None,
        'geometry': None,
        'lig': None,
        'ligocc': None,
        'spin': None,
        'charge': None
    }
    
    try:
        with open(molinp_path, 'r') as f:
            content = f.read()
        
        # Parse each target field
        for field in target_fields.keys():
            pattern = rf'-{field}\s+(\S+)'
            match = re.search(pattern, content)
            if match:
                target_fields[field] = match.group(1)
        
        return target_fields
        
    except (FileNotFoundError, UnicodeDecodeError):
        return target_fields

def extract_charge_spin(source_dir, dataset_info):
    """
    Enrich dataset_info with charge and spin information from .molinp files.

    Args:
        source_dir (str or Path): Directory containing .molinp files.
        dataset_info (list of dict): List of dictionaries, each with at least an 'id' key.

    Returns:
        list of dict: Updated dataset_info with additional fields from .molinp files.
    """

    source_path = Path(source_dir)
    molinp_files = list(source_path.rglob("*.molinp"))

    # Create a mapping of complex names to molinp files
    molinp_dict = {}
    for molinp_file in molinp_files:
        molinp_dict[molinp_file.stem] = molinp_file
    
    # Enrich each entry in dataset_info
    for i, info in enumerate(dataset_info):
        file_name = info['id']
        
        if file_name in molinp_dict:
            molinp_file = molinp_dict[file_name]
            molinp_data = parse_molinp_file(molinp_file)
            
            # Add molinp data to dataset_info
            dataset_info[i].update(molinp_data)
            print(f"Added molinp data for: {file_name}")
        else:
            print(f"No molinp file found for: {file_name}")
    
    return dataset_info

def extract_xyz_files(source_dir, destination_dir):
    """
    Copy all .xyz files from a source directory (and subdirectories) to a destination directory.
    Also appends file information to the global dataset_info list.

    Args:
        source_dir (str or Path): Directory to search for .xyz files.
        destination_dir (str or Path): Directory to copy .xyz files into.

    Returns:
        None. Files are copied and dataset_info is updated.
    """
    
    source_path = Path(source_dir)
    destination_path = Path(destination_dir)
    
    # Create destination directory if it doesn't exist
    destination_path.mkdir(parents=True, exist_ok=True)
    
    # Find all .xyz files
    xyz_files = list(source_path.rglob("*.xyz"))

    file_names = []
    
    print(f"Found {len(xyz_files)} .xyz files")
    
    for xyz_file in xyz_files:
        # Create destination filename (you can customize this)
        dest_file = destination_path / xyz_file.name
        
        # If file with same name exists, add parent folder name
        if dest_file.exists():
            parent_name = xyz_file.parent.name
            dest_file = destination_path / f"{parent_name}_{xyz_file.name}"
        
        # Copy the file
        shutil.copy2(xyz_file, dest_file)
        print(f"Copied: {xyz_file} -> {dest_file}")

        # Copy the files in a .csv file
        print('File without extension', xyz_file.stem)  # Output: Fe__2_5_nh3-6_5

        # add the name in a column 'id'
        dataset_info.append({
                'id': xyz_file.stem,
            })

In [ ]:
# Usage
source_directory = "Runs"
destination_directory = "extracted_xyz_files"
os.makedirs(destination_directory, exist_ok=True)

extract_xyz_files(source_directory, destination_directory)
dataset_info = extract_charge_spin(source_directory, dataset_info)

df = pd.DataFrame(dataset_info)
df.to_csv("general_info_xyz.csv")

The information obtained after executing the above python cells is stored in the csv file general_info_xyz.csv

### 2.2. Generating descriptors of our dataset
We calculate a variety of molecular descriptors.

- xyz2mol
- Morfeus steric descriptors:
- Electronic descriptors (XTB).

**NB!**
    📂 Data availability:
All descriptors are saved in CSV files in the [github repo](link)

In [20]:
# load the xyz file
xyz_dir = Path("extracted_xyz_files")
xyz_files = list(xyz_dir.glob("*.xyz"))

In [ ]:
# extract steric descriptors
all_results = []

for f in xyz_files:
    try:
        elements, coordinates = read_xyz(f)

        sterimol = Sterimol(elements, coordinates, 1, 2)  # atoms 1 and 2 (adjust indices as needed)
        L_value = sterimol.L_value
        B1_value = sterimol.B_1_value
        B5_value = sterimol.B_5_value

        buried_volume = BuriedVolume(elements, coordinates, 1)  # for atom 1
        bv = buried_volume.fraction_buried_volume

        cone_angle = ConeAngle(elements, coordinates, 1)
        ca_ca = cone_angle.cone_angle
        ca_tangent = cone_angle.tangent_atoms

        # Combine results
        result_steric = {
            "id": f.stem,
            "L_value": L_value,
            "B1_value": B1_value,
            "B5_value": B5_value,
            "Buriedvolume": bv,
            "Coneangle": ca_ca,
            "Tangentatoms": ca_tangent
        }
        
        all_results.append(result_steric)

    except Exception as e:
        print(f"Failed for {f}: {e}")

df = pd.DataFrame(all_results)
df.to_csv("morfeus_descriptors.csv", index=False)
df.head()


In [ ]:
# extract electronic descriptors
all_elec = []

for f in xyz_files:
    try:
        elements, coordinates = read_xyz(f)

        # xtb value
        xtb = XTB(elements, coordinates)
        print(xtb)
        
        xtb_ip = xtb.get_ip()
        xtb_ea = xtb.get_ea()
        xtb_homo = xtb.get_homo()
        xtb_get_charges = xtb.get_charges()
        xtb_get_bond_order = xtb.get_bond_order(1, 2)
        xtb_get_dipole = xtb.get_dipole()
        xtb_elec = xtb.get_global_descriptor("electrophilicity", corrected=True)
        xtb_nucl = xtb.get_global_descriptor("nucleophilicity", corrected=True)

        # Combine results
        result_electronic = {
            "id": f.stem,
            "IP": xtb_ip,
            "EA": xtb_ea,
            "HOMO": xtb_homo,
            "Charges": xtb_get_charges,
            "BondOrders": xtb_get_bond_order,
            "Dipole": xtb_get_dipole,
        }

        all_elec.append(result_electronic)

    except Exception as e:
        print(f"Failed for {f}: {e}")

df_elec = pd.DataFrame(all_elec)
df_elec.to_csv("xtb_descriptors.csv", index=False)
df_elec.head()

## Here !! 👋 👀
This is the starting point to run cells

### 2.3. Visualize dataset with the extracted descriptors: PCA

**NB!**
    📂 Data availability:
All descriptors are saved in CSV files in the [github repo](link)

In [20]:
# Load the data
xyz_dir = Path("extracted_xyz_files")
xyz_files = list(xyz_dir.glob("*.xyz"))

steric_df = pd.read_csv("morfeus_descriptors.csv")
general_info_df = pd.read_csv("general_info_xyz.csv") # Load the new file

In [ ]:
# Select only numeric columns for PCA and fill NaNs
numeric_cols = steric_df.select_dtypes(include='number').columns
X = steric_df[numeric_cols].fillna(0)

# Run PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Create a DataFrame for Plotly
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
# Add the 'id' column for hover information3
pca_df['id'] = steric_df['id'].values

# Create the interactive scatter plot
fig = px.scatter(pca_df, x='PC1', y='PC2', hover_data=['id'], 
                 title='PCA of Steric Descriptors')
fig.show()

# check if it's possible to separate the data by charge, as done later to include then more properties. 

In [22]:
# bonds are missing in
rdkit.Chem.Draw.IPythonConsole.ipython_3d = True  # enable py3Dmol inline visualization

In [ ]:
# merge all dataframes on the common key (probably 'id')
merged_df = steric_df.merge(general_info_df, on='id')

# prepare data for PCA
numeric_cols = merged_df.select_dtypes(include='number').columns
X = merged_df[numeric_cols].fillna(0) # Fill NaNs for PCA

# run PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# add PCA components to the main dataframe for plotting
merged_df['PC1'] = X_pca[:, 0]
merged_df['PC2'] = X_pca[:, 1]

# interactive plot
# Use 'color' for the metal core, 'symbol' for coordination number
fig = px.scatter(merged_df,
                 x='PC1',
                 y='PC2',
                 color='charge',          # Different colors for different metal cores
                 symbol='coord',        # Different markers for coordination numbers
                 # size='charge',       # You can also use size for charge (optional)
                 hover_data=['id'],     # Show the id on hover
                 #title='PCA Colored by Core, Shape by Coordination',
                 labels={
                     'PC1': f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)',
                     'PC2': f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)',
                     'core': 'Metal Core',
                     'coord': 'Coordination',
                     'charge': 'Charge'
                 })

# hover template to show more info
fig.update_traces(
    hovertemplate="<br>".join([
        "<b>%{customdata[0]}</b>",
        "Core: %{marker.color}",
        "Coord: %{marker.symbol}",
        "PC1: %{x:.3f}",
        "PC2: %{y:.3f}",
        "<extra></extra>"
    ])
)

# update layout for readability
fig.update_layout(
    #legend_title_text='Properties', # Better legend title
    width=900,  # Adjust size
    height=700,
    legend=dict(
        orientation="h",  # Horizontal orientation instead of vertical
        yanchor="bottom", # Anchor to the bottom
        y=-0.3,           # Position it below the plot
        xanchor="center", # Center horizontally
        x=0.5,
        title_text='Charge'  # Optional: set a title for the whole legend
)
)

fig.show()

# save the interactive plot
# fig.write_html("pca_analysis_interactive.html")

### 2.4. Searching specific metal complexes
If we are interesting in getting a metal complex with the lowest steric hindrance? How to look for it?  (steric_df)

In [ ]:
steric_property = 'Buriedvolume'
item_steric = steric_df[f'{steric_property}'].idxmax()
steric_df.iloc[steric_df[f'{steric_property}'].idxmax()]

In [25]:
print(steric_df.iloc[item_steric]['id'], ', ', item_steric)

fe_tdhd_2_pyridine_8_ammonia_0_cyanide_0_water_0_cyanide_0_water_0_ammonia_0_pyridine_0_s_5_conf_1 ,  33


In [ ]:
# visualize the molecule

from IPython.display import display

base_dir = Path("extracted_xyz_files")

target_filename = f"{steric_df.iloc[item_steric]['id']}.xyz"  # Change to your actual filename
file_path = base_dir / target_filename
print(file_path)

with open(file_path, 'r') as f:
    lines = f.readlines() 

# Check if the file exists
if file_path.exists():
    print(f"Found file: {file_path}")
    
    # Read and visualize the XYZ file
    with open(file_path, 'r') as f:
        xyz_content = f.read()
    
    viewer = py3Dmol.view(width=600, height=400)
    viewer.addModel(xyz_content, 'xyz')
    viewer.setStyle({'stick': {}})
    viewer.zoomTo()
    
    # Display the visualization
    display(viewer)
else:
    print(f"File not found: {file_path}")

Seaarch for the metal complex with the lowest dipole moment (electronic_df)